<a href="https://colab.research.google.com/github/martinlee192/AI-basic/blob/master/%EB%AF%B8%EB%8B%88%EB%B0%B0%EC%B9%98%EB%A5%BC_%ED%99%9C%EC%9A%A9%ED%95%9C_gradient_decent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Load Data**

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [4]:
torch.manual_seed(1)

In [5]:
import numpy as np

In [6]:
xy = np.loadtxt('/content/drive/MyDrive/DeepLearningExample/data-01-test-score.csv', delimiter=',', dtype=np.float32)


In [11]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
print(y_data)

[[152.]
 [185.]
 [180.]
 [196.]
 [142.]
 [101.]
 [149.]
 [115.]
 [175.]
 [164.]
 [141.]
 [141.]
 [184.]
 [152.]
 [148.]
 [192.]
 [147.]
 [183.]
 [177.]
 [159.]
 [177.]
 [175.]
 [175.]
 [149.]
 [192.]]


**복습 1**

In [14]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 모델 초기화
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

#optimizer 설정
optimizer = optim.SGD([W,b], lr= 1e-5)

nb_epoch = 20
for epoch in range(nb_epoch +1) :

    #H(x) 계산
    hypothesis = x_train.matmul(W) + b

    #Cost 계산
    cost = torch.mean((hypothesis - y_train)**2)

    #Cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epoch, cost.item()))


Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 9920.530273
Epoch    2/20 Cost: 3675.298340
Epoch    3/20 Cost: 1366.260498
Epoch    4/20 Cost: 512.542480
Epoch    5/20 Cost: 196.896637
Epoch    6/20 Cost: 80.190987
Epoch    7/20 Cost: 37.038696
Epoch    8/20 Cost: 21.081343
Epoch    9/20 Cost: 15.178760
Epoch   10/20 Cost: 12.993679
Epoch   11/20 Cost: 12.183023
Epoch   12/20 Cost: 11.880535
Epoch   13/20 Cost: 11.765958
Epoch   14/20 Cost: 11.720851
Epoch   15/20 Cost: 11.701438
Epoch   16/20 Cost: 11.691514
Epoch   17/20 Cost: 11.685116
Epoch   18/20 Cost: 11.680005
Epoch   19/20 Cost: 11.675380
Epoch   20/20 Cost: 11.670952


**복습 2** High-level implementation with nn.module

In [18]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)

    def forward(self, x):
        return self.linear(x)
        



In [20]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 22444.050781
Epoch    1/20 Cost: 8304.253906
Epoch    2/20 Cost: 3076.377197
Epoch    3/20 Cost: 1143.485474
Epoch    4/20 Cost: 428.840912
Epoch    5/20 Cost: 164.614960
Epoch    6/20 Cost: 66.922005
Epoch    7/20 Cost: 30.800661
Epoch    8/20 Cost: 17.444128
Epoch    9/20 Cost: 12.504412
Epoch   10/20 Cost: 10.676523
Epoch   11/20 Cost: 9.999249
Epoch   12/20 Cost: 9.747339
Epoch   13/20 Cost: 9.652740
Epoch   14/20 Cost: 9.616273
Epoch   15/20 Cost: 9.601300
Epoch   16/20 Cost: 9.594303
Epoch   17/20 Cost: 9.590237
Epoch   18/20 Cost: 9.587271
Epoch   19/20 Cost: 9.584668
Epoch   20/20 Cost: 9.582247


Dataset and DataLoader

In [21]:
from torch.utils.data import Dataset, DataLoader

In [34]:
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                       [93, 88, 93],
                       [89, 91, 90],
                       [96, 98, 100],
                       [73, 66, 70],
                       [53, 46, 55],
                       [69, 74, 77]]
        self.y_data = [[152], [185], [180], [196], [142],[101],[149]]

    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self,idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])

        return x,y


In [37]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=4 , shuffle=True)


Dataloader를 이용해 미니배치 그레디언트 해보기


In [38]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr= 1e-5)

nb_epochs = 20
for epoch in range(nb_epochs +1):
    for batch_idx, sample in enumerate(dataloader):
        x_train, y_train = sample

        #H(x)
        prediction = model(x_train)

        #cost
        cost = F.mse_loss(prediction, y_train)

        #cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/20 Batch 1/2 Cost: 12460.433594
Epoch    0/20 Batch 2/2 Cost: 3495.104736
Epoch    1/20 Batch 1/2 Cost: 1713.395142
Epoch    1/20 Batch 2/2 Cost: 650.336609
Epoch    2/20 Batch 1/2 Cost: 306.852966
Epoch    2/20 Batch 2/2 Cost: 54.067005
Epoch    3/20 Batch 1/2 Cost: 34.876091
Epoch    3/20 Batch 2/2 Cost: 23.002028
Epoch    4/20 Batch 1/2 Cost: 8.139425
Epoch    4/20 Batch 2/2 Cost: 3.286049
Epoch    5/20 Batch 1/2 Cost: 5.281459
Epoch    5/20 Batch 2/2 Cost: 1.616291
Epoch    6/20 Batch 1/2 Cost: 0.593210
Epoch    6/20 Batch 2/2 Cost: 6.846048
Epoch    7/20 Batch 1/2 Cost: 1.958618
Epoch    7/20 Batch 2/2 Cost: 3.013086
Epoch    8/20 Batch 1/2 Cost: 3.048981
Epoch    8/20 Batch 2/2 Cost: 1.555069
Epoch    9/20 Batch 1/2 Cost: 3.065165
Epoch    9/20 Batch 2/2 Cost: 1.537583
Epoch   10/20 Batch 1/2 Cost: 2.754077
Epoch   10/20 Batch 2/2 Cost: 2.094598
Epoch   11/20 Batch 1/2 Cost: 2.198039
Epoch   11/20 Batch 2/2 Cost: 2.746386
Epoch   12/20 Batch 1/2 Cost: 2.801709
Epoch   